In [2]:
#!/usr/bin/env python3
###-----------------------------------------------------------##
#@author narumeena
#@description -applying bayesian optimization on random forest 
#             -using hyperopt library 
###-----------------------------------------------------------##

In [3]:
#import libraries 
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import numpy as np # linear algebra
from sklearn.model_selection import train_test_split # split the dataset into traninig and testing dataset 

from xgboost import XGBClassifier
from sklearn.metrics import roc_curve, auc                # to import roc curve abd auc metrics for evaluation 
#from sklearn.grid_search import GridSearchCV              # grid search is used for hyperparameters-optimization
from sklearn.model_selection import KFold                # cross validation using the kfold algorithm
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials     # library for hyperparameters-optimization
from sklearn.model_selection import cross_val_score

import seaborn as sns                                     # Python graphing library
import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
#import data from filterd dataset with maximum 30% files 
filtredAttributes = pd.read_csv("../analysis/trainingDataSet/case_control_filtred.csv", sep=",")
filtredAttributes.shape

(138665, 74)

In [5]:
filtredAttributes.head()

,GC,CpG,minDistTSS,minDistTSE,priPhCons,mamPhCons,verPhCons,priPhyloP,mamPhyloP,verPhyloP,...,Consequence_DOWNSTREAM,Consequence_INTRONIC,Consequence_NONCODING_CHANGE,Consequence_NON_SYNONYMOUS,Consequence_REGULATORY,Consequence_SPLICE_SITE,Consequence_STOP_GAINED,Consequence_STOP_LOST,Consequence_SYNONYMOUS,Consequence_UPSTREAM
0,0.715,0.253,6715,185,0.011,0.001,0.0,-0.646,0.096,0.126,...,0,0,0,0,0,0,1,0,0,0
1,0.715,0.253,6715,185,0.011,0.001,0.0,-0.646,0.096,0.126,...,0,0,0,0,1,0,0,0,0,0
2,0.609,0.120,212,2176,0.039,0.984,1.0,0.595,4.158,5.651,...,0,0,0,1,0,0,0,0,0,0
3,0.609,0.120,212,2176,0.039,0.984,1.0,0.595,4.158,5.651,...,0,0,0,0,0,0,0,0,0,1
4,0.675,0.213,814,1969,0.192,0.999,1.0,-1.149,0.850,3.250,...,0,0,0,1,0,0,0,0,0,0


In [6]:
pd.options.display.max_rows = 4000
filtredAttributes.isnull().sum()

GC                                  0
CpG                                 0
minDistTSS                          0
minDistTSE                          0
priPhCons                         143
mamPhCons                         143
verPhCons                         143
priPhyloP                         143
mamPhyloP                         143
verPhyloP                         143
bStatistic                       1058
cHmm_E1                             0
cHmm_E2                             0
cHmm_E3                             0
cHmm_E4                             0
cHmm_E5                             0
cHmm_E6                             0
cHmm_E7                             0
cHmm_E8                             0
cHmm_E9                             0
cHmm_E10                            0
cHmm_E11                            0
cHmm_E12                            0
cHmm_E13                            0
cHmm_E14                            0
cHmm_E15                            0
cHmm_E16    

In [7]:
#missing values 

filtredAttributes['priPhCons'].fillna(0.115,inplace=True)
filtredAttributes['mamPhCons'].fillna(0.079,inplace=True)
filtredAttributes['verPhCons'].fillna(0.094,inplace=True)
filtredAttributes['priPhyloP'].fillna(-0.033,inplace=True)
filtredAttributes['mamPhyloP'].fillna(-0.038,inplace=True)
filtredAttributes['verPhyloP'].fillna(0.017,inplace=True)
filtredAttributes['bStatistic'].fillna(800.261,inplace=True)
filtredAttributes['minDistTSS'].fillna(10000000,inplace=True)
filtredAttributes['minDistTSE'].fillna(10000000,inplace=True)
filtredAttributes['RemapOverlapTF'].fillna(0.5,inplace=True)
filtredAttributes['EncodeH3K4me1-max'].fillna(0,inplace=True)
filtredAttributes['EncodeH3K4me2-max'].fillna(0,inplace=True)
filtredAttributes['EncodeH3K4me3-max'].fillna(0,inplace=True)
filtredAttributes['EncodeH3K9ac-max'].fillna(0,inplace=True)
filtredAttributes['EncodeH3K9me3-max'].fillna(0,inplace=True)
filtredAttributes['EncodeH3K27ac-max'].fillna(0,inplace=True)
filtredAttributes['EncodeH3K27me3-max'].fillna(0,inplace=True)
filtredAttributes['EncodeH3K36me3-max'].fillna(0,inplace=True)
filtredAttributes['EncodeH3K79me2-max'].fillna(0,inplace=True)
filtredAttributes['EncodeH4K20me1-max'].fillna(0,inplace=True)
filtredAttributes['EncodeH2AFZ-max'].fillna(0,inplace=True)
filtredAttributes['EncodetotalRNA-max'].fillna(0,inplace=True)
filtredAttributes['EncodeDNase-max'].fillna(0,inplace=True)

In [8]:
pd.options.display.max_rows = 4000
filtredAttributes.isnull().sum()

GC                              0
CpG                             0
minDistTSS                      0
minDistTSE                      0
priPhCons                       0
mamPhCons                       0
verPhCons                       0
priPhyloP                       0
mamPhyloP                       0
verPhyloP                       0
bStatistic                      0
cHmm_E1                         0
cHmm_E2                         0
cHmm_E3                         0
cHmm_E4                         0
cHmm_E5                         0
cHmm_E6                         0
cHmm_E7                         0
cHmm_E8                         0
cHmm_E9                         0
cHmm_E10                        0
cHmm_E11                        0
cHmm_E12                        0
cHmm_E13                        0
cHmm_E14                        0
cHmm_E15                        0
cHmm_E16                        0
cHmm_E17                        0
cHmm_E18                        0
cHmm_E19      

In [9]:
# One-hot encode the data using pandas get_dummies
filtredAttributes = pd.get_dummies(filtredAttributes)

In [10]:
# Labels are the values we want to predict
labels = filtredAttributes['category']
labels

0         1
1         1
2         1
3         1
4         1
         ..
138660    0
138661    0
138662    0
138663    0
138664    0
Name: category, Length: 138665, dtype: int64

In [11]:
# Remove the labels from the features
# axis 1 refers to the columns
features= filtredAttributes.drop('category', axis = 1)
features.head()

,GC,CpG,minDistTSS,minDistTSE,priPhCons,mamPhCons,verPhCons,priPhyloP,mamPhyloP,verPhyloP,...,Consequence_DOWNSTREAM,Consequence_INTRONIC,Consequence_NONCODING_CHANGE,Consequence_NON_SYNONYMOUS,Consequence_REGULATORY,Consequence_SPLICE_SITE,Consequence_STOP_GAINED,Consequence_STOP_LOST,Consequence_SYNONYMOUS,Consequence_UPSTREAM
0,0.715,0.253,6715,185,0.011,0.001,0.0,-0.646,0.096,0.126,...,0,0,0,0,0,0,1,0,0,0
1,0.715,0.253,6715,185,0.011,0.001,0.0,-0.646,0.096,0.126,...,0,0,0,0,1,0,0,0,0,0
2,0.609,0.120,212,2176,0.039,0.984,1.0,0.595,4.158,5.651,...,0,0,0,1,0,0,0,0,0,0
3,0.609,0.120,212,2176,0.039,0.984,1.0,0.595,4.158,5.651,...,0,0,0,0,0,0,0,0,0,1
4,0.675,0.213,814,1969,0.192,0.999,1.0,-1.149,0.850,3.250,...,0,0,0,1,0,0,0,0,0,0


In [12]:
# Split the data into training and testing sets
X_train, X_test, Y_train, Y_test = train_test_split(features, labels, test_size = 0.25, random_state = 100)

In [1]:
X_train.head()

NameError: name 'X_train' is not defined

In [14]:
Y_train

42192    1
46590    1
78842    0
48882    1
33305    1
        ..
82270    0
65615    0
77655    0
56088    1
38408    1
Name: category, Length: 103998, dtype: int64

In [15]:
#Function for model performance using AUC
def performance(Model,Y,X):
    # Perforamnce of the model
    fpr, tpr, _ = roc_curve(Y, Model.predict_proba(X)[:,1])
    AUC  = auc(fpr, tpr)
    print ('the AUC is : %0.4f' %  AUC)
    plt.figure()
    plt.plot(fpr, tpr, label='ROC curve (area = %0.4f)' % AUC)
    plt.plot([0, 1], [0, 1], 'k--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC Curve')
    plt.legend(loc="lower right")
    plt.show()

# CNN without hyper parameter optimisation



In [16]:
import tensorflow as tf
with tf.device('/gpu:0'):
  a = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[2, 3], name='a')
  b = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], shape=[3, 2], name='b')
  c = tf.matmul(a, b)
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))
print(sess.run(c))

/home/narumeena/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/narumeena/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/narumeena/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/narumeena/.local/lib/python3.6/site-packages/tensorflow/python/framework/dty

InvalidArgumentError: Cannot assign a device for operation MatMul: node MatMul (defined at <ipython-input-16-35621bae5838>:5) was explicitly assigned to /device:GPU:0 but available devices are [ /job:localhost/replica:0/task:0/device:CPU:0, /job:localhost/replica:0/task:0/device:XLA_CPU:0, /job:localhost/replica:0/task:0/device:XLA_GPU:0 ]. Make sure the device specification refers to a valid device.
	 [[MatMul]]

Errors may have originated from an input operation.
Input Source operations connected to node MatMul:
 b (defined at <ipython-input-16-35621bae5838>:4)	
 a (defined at <ipython-input-16-35621bae5838>:3)

In [4]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 6064877345647891603
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 3478342581889980517
physical_device_desc: "device: XLA_GPU device"
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 13288259363879900371
physical_device_desc: "device: XLA_CPU device"
]


In [5]:

from keras import backend as K
K.tensorflow_backend._get_available_gpus()

[]

In [1]:
import torch
import torchvision
import torchvision.transforms as transforms

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# Assuming that we are on a CUDA machine, this should print a CUDA device:

print(device)

cuda:0


In [3]:
net.to(device)

NameError: name 'net' is not defined

In [ ]:
import keras
from keras.models import Sequential,Input,Model
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.layers.advanced_activations import LeakyReLU

In [13]:
batch_size = 64
epochs = 20
num_classes = 10

In [14]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),activation='linear',input_shape=(28,28,1),padding='same'))
model.add(LeakyReLU(alpha=0.1))
model.add(MaxPooling2D((2, 2),padding='same'))
model.add(Conv2D(64, (3, 3), activation='linear',padding='same'))
model.add(LeakyReLU(alpha=0.1))
model.add(MaxPooling2D(pool_size=(2, 2),padding='same'))
model.add(Conv2D(128, (3, 3), activation='linear',padding='same'))
model.add(LeakyReLU(alpha=0.1))                  
model.add(MaxPooling2D(pool_size=(2, 2),padding='same'))
model.add(Flatten())
model.add(Dense(128, activation='linear'))
model.add(LeakyReLU(alpha=0.1))                  
model.add(Dense(num_classes, activation='softmax'))

In [15]:
model.compile(loss=keras.losses.categorical_crossentropy, optimizer=keras.optimizers.Adam(),metrics=['accuracy'])

In [16]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 28, 28, 32)        320       
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 28, 28, 32)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 14, 14, 64)        18496     
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 14, 14, 64)        0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 7, 7, 64)          0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 7, 7, 128)        

In [28]:
eval_set = [(X_train, Y_train), (X_test, Y_test)]
eval_metric = ["auc","error"]
%time model.fit(X_train, Y_train, eval_metric=eval_metric, eval_set=eval_set, verbose=True)


[0]	validation_0-auc:0.991128	validation_0-error:0.028154	validation_1-auc:0.990936	validation_1-error:0.029596
[1]	validation_0-auc:0.991921	validation_0-error:0.028049	validation_1-auc:0.991723	validation_1-error:0.029654
[2]	validation_0-auc:0.992804	validation_0-error:0.028126	validation_1-auc:0.992778	validation_1-error:0.029625
[3]	validation_0-auc:0.992913	validation_0-error:0.027789	validation_1-auc:0.9929	validation_1-error:0.02948
[4]	validation_0-auc:0.993204	validation_0-error:0.027212	validation_1-auc:0.993208	validation_1-error:0.028961
[5]	validation_0-auc:0.993242	validation_0-error:0.027251	validation_1-auc:0.993272	validation_1-error:0.028904
[6]	validation_0-auc:0.994041	validation_0-error:0.026664	validation_1-auc:0.994145	validation_1-error:0.028327
[7]	validation_0-auc:0.994057	validation_0-error:0.026385	validation_1-auc:0.994155	validation_1-error:0.028125
[8]	validation_0-auc:0.994323	validation_0-error:0.025587	validation_1-auc:0.994383	validation_1-error:0.02

[73]	validation_0-auc:0.998176	validation_0-error:0.016395	validation_1-auc:0.998119	validation_1-error:0.018808
[74]	validation_0-auc:0.998316	validation_0-error:0.016212	validation_1-auc:0.998179	validation_1-error:0.018634
[75]	validation_0-auc:0.998382	validation_0-error:0.01601	validation_1-auc:0.998216	validation_1-error:0.018375
[76]	validation_0-auc:0.998421	validation_0-error:0.016048	validation_1-auc:0.998242	validation_1-error:0.018433
[77]	validation_0-auc:0.998447	validation_0-error:0.01601	validation_1-auc:0.99827	validation_1-error:0.018433
[78]	validation_0-auc:0.998474	validation_0-error:0.015933	validation_1-auc:0.998285	validation_1-error:0.018346
[79]	validation_0-auc:0.998487	validation_0-error:0.015923	validation_1-auc:0.998287	validation_1-error:0.018231
[80]	validation_0-auc:0.998513	validation_0-error:0.015875	validation_1-auc:0.99831	validation_1-error:0.018144
[81]	validation_0-auc:0.998535	validation_0-error:0.015837	validation_1-auc:0.998336	validation_1-er

[146]	validation_0-auc:0.99929	validation_0-error:0.011731	validation_1-auc:0.999046	validation_1-error:0.01425
[147]	validation_0-auc:0.999294	validation_0-error:0.011712	validation_1-auc:0.99905	validation_1-error:0.014192
[148]	validation_0-auc:0.999297	validation_0-error:0.011683	validation_1-auc:0.999053	validation_1-error:0.014221
[149]	validation_0-auc:0.999309	validation_0-error:0.011587	validation_1-auc:0.999069	validation_1-error:0.014048
[150]	validation_0-auc:0.999332	validation_0-error:0.011433	validation_1-auc:0.999098	validation_1-error:0.013731
[151]	validation_0-auc:0.999337	validation_0-error:0.011375	validation_1-auc:0.999101	validation_1-error:0.013558
[152]	validation_0-auc:0.999339	validation_0-error:0.011318	validation_1-auc:0.999102	validation_1-error:0.013529
[153]	validation_0-auc:0.999343	validation_0-error:0.011279	validation_1-auc:0.999104	validation_1-error:0.013471
[154]	validation_0-auc:0.999358	validation_0-error:0.011144	validation_1-auc:0.999123	valid

[219]	validation_0-auc:0.999609	validation_0-error:0.008423	validation_1-auc:0.999381	validation_1-error:0.010673
[220]	validation_0-auc:0.99961	validation_0-error:0.008414	validation_1-auc:0.999382	validation_1-error:0.010644
[221]	validation_0-auc:0.999611	validation_0-error:0.008414	validation_1-auc:0.999382	validation_1-error:0.010615
[222]	validation_0-auc:0.999614	validation_0-error:0.008385	validation_1-auc:0.999386	validation_1-error:0.010558
[223]	validation_0-auc:0.999616	validation_0-error:0.008394	validation_1-auc:0.999388	validation_1-error:0.010471
[224]	validation_0-auc:0.99962	validation_0-error:0.008327	validation_1-auc:0.999392	validation_1-error:0.010586
[225]	validation_0-auc:0.999623	validation_0-error:0.008269	validation_1-auc:0.999395	validation_1-error:0.010558
[226]	validation_0-auc:0.999623	validation_0-error:0.008269	validation_1-auc:0.999396	validation_1-error:0.010558
[227]	validation_0-auc:0.999626	validation_0-error:0.008212	validation_1-auc:0.999399	vali

[292]	validation_0-auc:0.999725	validation_0-error:0.006673	validation_1-auc:0.999494	validation_1-error:0.009577
[293]	validation_0-auc:0.999727	validation_0-error:0.006644	validation_1-auc:0.999496	validation_1-error:0.009519
[294]	validation_0-auc:0.99973	validation_0-error:0.006625	validation_1-auc:0.999496	validation_1-error:0.00949
[295]	validation_0-auc:0.999734	validation_0-error:0.006606	validation_1-auc:0.999502	validation_1-error:0.009404
[296]	validation_0-auc:0.999736	validation_0-error:0.006577	validation_1-auc:0.999503	validation_1-error:0.009317
[297]	validation_0-auc:0.999736	validation_0-error:0.006567	validation_1-auc:0.999503	validation_1-error:0.009288
[298]	validation_0-auc:0.999738	validation_0-error:0.006577	validation_1-auc:0.999504	validation_1-error:0.009288
[299]	validation_0-auc:0.999744	validation_0-error:0.006452	validation_1-auc:0.999517	validation_1-error:0.009115
CPU times: user 1min 24s, sys: 320 ms, total: 1min 24s
Wall time: 1min 24s


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.05, max_delta_step=0, max_depth=5,
              min_child_weight=1, missing=None, n_estimators=300, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [18]:
train = model.fit(X_train, Y_train, batch_size=batch_size,epochs=epochs,verbose=1,validation_data=(X_test, Y_test))

ValueError: Error when checking input: expected conv2d_1_input to have 4 dimensions, but got array with shape (103998, 73)

# CNN using hyperopt

In [ ]:
def acc_model(params):
    clf = XGBClassifier(**params)
    return cross_val_score(clf, X_train, Y_train).mean()

param_space = {
    'max_depth': hp.choice('max_depth',range(1,20)),
    'learning_rate': hp.choice('learning_rate', np.arange(0.01,0.1,0.01)),
    'n_estimators': hp.choice('n_estimators', range(100,500)),
    'subsample': hp.choice('subsample', np.arange(0.4,1,0.1)),
    'colsample_bytree': hp.choice('colsample_bytree', np.arange(0.3,0.8,0.1))}

best = 0
def f(params):
    global best
    acc = acc_model(params)
    if acc > best:
        best = acc
    print ('new best:', best, params)
    return {'loss': -acc, 'status': STATUS_OK}

trials = Trials()
best = fmin(f, param_space, algo=tpe.suggest, max_evals=100, trials=trials)
print ('best:')
print (best)


new best:                                              
0.973566803322645                                      
{'colsample_bytree': 0.6000000000000001, 'learning_rate': 0.03, 'max_depth': 1, 'n_estimators': 470, 'subsample': 0.7}
  1%|          | 1/100 [01:40<2:46:23, 100.85s/trial, best loss: -0.973566803322645]

In [ ]:
rf1=RandomForestClassifier(max_features=113, n_estimators=498, criterion= 'entropy', max_depth=2,random_state=1)

In [ ]:
rf1.fit(X=X_train,y=Y_train)

In [ ]:
performance(Model=rf1,Y=Y_test,X=X_test)

# calculating classification accuracy